In [ ]:
# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum_output = 0
for sqrt_value in generator:
    print(sqrt_value)
    sum_output += sqrt_value

print("The sum of the outputs for limit = 5 is ", sum_output)
    


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
The sum of the outputs for limit = 5 is  8.382332347441762


In [2]:
limit=13
generator = square_root_generator(limit)

for sqrt_value in generator:
    pass
print("sqrt_value is {} for limit {} ".format(sqrt_value, limit))


sqrt_value is 3.605551275463989 for limit 13 


In [ ]:
# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**


In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data = []
for person in people_1():
    # print(person)
    data.append(person)
print(data)
print(type(data))

In [1]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [1]:
!pip install duckdb --pre --upgrade


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# 1 of the homework steps for DuckDB
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data = []
for person in people_1():
    # print(person)
    data.append(person)
print(data)
print(type(data))

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(data,
										table_name="people_data",
										write_disposition="replace",
                    primary_key='record_hash')

# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb", read_only=False)

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n Rides table below: Note the times are properly typed")
rides = conn.sql("SELECT * FROM people_data").df()
display(rides)


[{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}, {'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}, {'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}, {'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}, {'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}]
<class 'list'>


2024-02-18 10:58:11,734|[WARNING              ]|23892|15644|dlt|reference.py|_verify_schema:357|A column record_hash in table people_data in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_data         │
└─────────────────────┘




 Rides table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708234091.1631846,mHMpgP/fDb4vyw
1,2,Person_2,27,City_A,1708234091.1631846,RfZmxDnAuIzMCw
2,3,Person_3,28,City_A,1708234091.1631846,oPiADjlz/X7VDw
3,4,Person_4,29,City_A,1708234091.1631846,z/GPLVDZz4lhiQ
4,5,Person_5,30,City_A,1708234091.1631846,8PbdMi3NfsXsbw


In [5]:
print("\n\n\n Sum of all ages from initial load")

all = conn.sql("""
SELECT *
FROM people_data
""").df()
display(all)

res = conn.sql("""
SELECT sum(age)
FROM people_data
""").df()
display(res)




 Sum of all ages from initial load


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708233558.0566447,u1HreLlabBRmdg
1,2,Person_2,27,City_A,1708233558.0566447,UPX376e7pUuaWg
2,3,Person_3,28,City_A,1708233558.0566447,DojP189BWGUXLw
3,4,Person_4,29,City_A,1708233558.0566447,hVKktn2BqX6Gag
4,5,Person_5,30,City_A,1708233558.0566447,fF6cD7M/3b3JQA


,sum(age)
0,140.0


In [6]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

second = []
for person in people_2():
    print(person)
    second.append(person)
print(second)
print(type(second))


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
[{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}, {'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}, {'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}, {'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}, {'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}, {'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}]
<class 'l

In [8]:
# 1 of the homework steps for DuckDB
# run the pipeline with default settings, and capture the outcome
info = pipeline.run(second,
										table_name="people_data",
										write_disposition="append",
                    primary_key='record_hash')


2024-02-18 11:01:45,014|[WARNING              ]|23892|15644|dlt|reference.py|_verify_schema:357|A column record_hash in table people_data in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


In [9]:
print("\n\n\n Sum of all ages from after append of second generator to the first generator in the table")

all = conn.sql("""
SELECT *
FROM people_data
""").df()
display(all)

res = conn.sql("""
SELECT sum(age)
FROM people_data
""").df()
display(res)




 Sum of all ages from after append of second generator to the first generator in the table


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708234091.1631846,mHMpgP/fDb4vyw,None
1,2,Person_2,27,City_A,1708234091.1631846,RfZmxDnAuIzMCw,None
2,3,Person_3,28,City_A,1708234091.1631846,oPiADjlz/X7VDw,None
3,4,Person_4,29,City_A,1708234091.1631846,z/GPLVDZz4lhiQ,None
4,5,Person_5,30,City_A,1708234091.1631846,8PbdMi3NfsXsbw,None
5,3,Person_3,33,City_B,1708234304.419378,L+cyZA3fXncu1w,Job_3
6,4,Person_4,34,City_B,1708234304.419378,6mNfbwwWvLWG2A,Job_4
7,5,Person_5,35,City_B,1708234304.419378,RvjYJfx+qVgSiA,Job_5
8,6,Person_6,36,City_B,1708234304.419378,H/+t+t1JyUUluw,Job_6
9,7,Person_7,37,City_B,1708234304.419378,U5oiTeZOt8Pzqw,Job_7


,sum(age)
0,353.0


In [11]:
# 3. Merge a generator

# '''Re-use the generators from Exercise 2.

# A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

# Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

# After loading, you should have a total of 8 records, and ID 3 should have age 33.

# Question: **Calculate the sum of ages of all the people loaded as described above.**'''


In [13]:
# Since the connections are already open in DuckDBm, I will re-use them. Also the data built by generators 1 and 2.

# run the pipeline with default settings, and capture the outcome
# Use the output from first generator
info = pipeline.run(data,
										table_name="people_data",
										write_disposition="replace",
                    primary_key='id') # Make the ID column as the primary key to support updates to the table based on ID

# Use the output from second generator
info = pipeline.run(second,
										table_name="people_data",
										write_disposition="merge",
                    primary_key='id') # Make the ID column as the primary key to support updates to the table based on ID

print("\n\n\n Sum of all ages from after MERGE")

all = conn.sql("""
SELECT *
FROM people_data
order by id
""").df()
display(all)

res = conn.sql("""
SELECT sum(age)
FROM people_data
""").df()
display(res)



2024-02-18 11:12:20,171|[WARNING              ]|23892|15644|dlt|reference.py|_verify_schema:357|A column record_hash in table people_data in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-18 11:12:21,146|[WARNING              ]|23892|15644|dlt|reference.py|_verify_schema:357|A column record_hash in table people_data in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-18 11:12:21,153|[WARNING              ]|23892|15644|dlt|reference.py|_verify_schema:357|A column record_hash in table people_data in schema dlt_ipykernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key




 Sum of all ages from after MERGE


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708234939.6013849,PKgqq4+Mts7lKg,None
1,2,Person_2,27,City_A,1708234939.6013849,OHchhKZbgeWqag,None
2,3,Person_3,33,City_B,1708234940.6256657,/HWlFJ1J2Wh7Ug,Job_3
3,4,Person_4,34,City_B,1708234940.6256657,dhE4rLf0w8YPXA,Job_4
4,5,Person_5,35,City_B,1708234940.6256657,9Q1m5MqX2ZsKJA,Job_5
5,6,Person_6,36,City_B,1708234940.6256657,6c6q9X8u8FvYWw,Job_6
6,7,Person_7,37,City_B,1708234940.6256657,DGUCoXrrh9C/ow,Job_7
7,8,Person_8,38,City_B,1708234940.6256657,oNu9vsQJ9lrzBg,Job_8


,sum(age)
0,266.0
